In [270]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, TargetEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

import torch
from torch import nn

In [280]:
path = 'playground-series-s5e11_unzipped/train.csv'


In [251]:
train_data = pd.read_csv(path)

In [252]:
train_data

,id,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
593989,593989,23004.26,0.152,703,20958.37,10.92,Female,Single,High School,Employed,Business,C3,1.0
593990,593990,35289.43,0.105,559,3257.24,14.62,Male,Single,Bachelor's,Employed,Debt consolidation,F5,1.0
593991,593991,47112.64,0.072,675,929.27,14.13,Female,Married,Bachelor's,Employed,Debt consolidation,C1,1.0
593992,593992,76748.44,0.067,740,16290.40,9.87,Male,Single,Bachelor's,Employed,Debt consolidation,B2,1.0


In [282]:
data = train_data
data_copy = data.copy()

# data_copy.loc[data_copy['gender'] == 'Male']['gender'] = 1
# data_copy.loc[data_copy['gender'] == 'Female']['gender'] = 0
# data_copy['gender'][data_copy['gender'] == 'Male'] = np.float32(1) ;
# data_copy['gender'][data_copy['gender'] == 'Female'] = np.float32(0);
label_encoder_gen = LabelEncoder()
data_copy['gender'] = label_encoder_gen.fit_transform(data_copy['gender'])

label_encoder_marstatus = LabelEncoder()
data_copy['marital_status'] = label_encoder_marstatus.fit_transform(data_copy['marital_status'])

# data_copy['marital_status'][data_copy['marital_status'] == 'Single'] = np.float32(0) ;
# data_copy['marital_status'][data_copy['marital_status'] == 'Married'] = np.float32(1);
# data_copy['marital_status'][data_copy['marital_status'] == 'Divorced'] = np.float32(2);

lable_encoder_empstatus = LabelEncoder()
data_copy['employment_status'] = lable_encoder_empstatus.fit_transform(data_copy['employment_status'])
# data_copy['employment_status'][data_copy['employment_status'] == 'Self-employed'] = np.float32(0);
# data_copy['employment_status'][data_copy['employment_status'] == 'Employed'] = np.float32(1);
# data_copy['employment_status'][data_copy['employment_status'] == 'Unemployed'] = np.float32(2);

lable_encoder_loanpurp = LabelEncoder()
lable_encoder_loanpurp.fit(data_copy['loan_purpose'])
data_copy['loan_purpose'] = lable_encoder_loanpurp.transform(data_copy['loan_purpose'])
data_copy['loan_purpose'] = data_copy['loan_purpose'].apply(lambda x:np.float32(x))


ordinal_encoder_edu = OrdinalEncoder()
data_copy['education_level'] = ordinal_encoder_edu.fit_transform(pd.DataFrame(data_copy['education_level']))

ordinal_encoder_grade = OrdinalEncoder()
data_copy['grade_subgrade'] = ordinal_encoder_grade.fit_transform(pd.DataFrame(data_copy['grade_subgrade']))

data_copy.drop(columns=['id'], inplace=True);
data_copy.drop(columns = ['loan_paid_back'], inplace=True)

In [283]:
stan_scale = StandardScaler();
stan_scale.fit(data_copy)
data_copy_scaled = stan_scale.transform(data_copy)

In [284]:
data['loan_paid_back'].value_counts()

loan_paid_back
1.0    474494
0.0    119500
Name: count, dtype: int64

In [285]:
data_copy.sample(10)

,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade
512580,26452.02,0.053,662,14809.12,15.25,0,1,2.0,2,2.0,19.0
131032,9685.72,0.176,685,9685.62,10.00,1,3,0.0,3,2.0,13.0
348149,86522.99,0.081,697,14884.13,11.07,0,2,0.0,0,0.0,12.0
61050,39196.98,0.151,633,24361.86,15.62,0,1,0.0,0,1.0,15.0
567838,41513.45,0.183,590,15291.75,13.86,1,1,1.0,3,2.0,23.0
578029,48842.73,0.181,658,18266.94,14.76,0,2,0.0,3,2.0,19.0
530095,12287.25,0.265,696,18305.23,10.66,1,2,0.0,0,7.0,13.0
93670,26391.79,0.111,685,1468.28,13.25,0,2,1.0,0,2.0,11.0
521306,53925.76,0.172,644,9525.55,14.86,0,2,1.0,0,2.0,19.0
98469,43446.12,0.077,703,16686.17,10.73,2,2,1.0,0,1.0,12.0


In [73]:
x_train

array([[-1.24282768, -0.19972658, -0.17890889, ..., -0.36823473,
        -0.58013597,  0.50184383],
       [ 0.1655245 ,  1.52106229, -0.62996962, ..., -0.36823473,
         0.59276219, -1.99265177],
       [-0.12129765, -0.65179822,  0.57887313, ..., -0.36823473,
        -0.18916991,  0.50184383],
       ...,
       [ 0.84927768,  0.45650646, -1.22536979, ...,  2.04651483,
         0.78824522,  0.50184383],
       [ 0.13032897, -0.8267937 , -1.64034566, ...,  0.23545266,
         1.96114338, -1.99265177],
       [ 2.12472663, -0.50596866,  0.61495799, ..., -0.36823473,
        -0.775619  ,  0.50184383]], shape=(593, 12))

In [262]:
x_train, x_test, y_train, y_test = train_test_split(
    data_copy_scaled,
    data['loan_paid_back'],
    test_size=.5,
    # random_state=4,
    shuffle=True,
);

In [389]:
x_tensor = torch.tensor(data_copy_scaled, dtype=torch.float32)
y_tensor =  torch.tensor(data['loan_paid_back'].to_numpy(), dtype=torch.float32)
x_train_tensor, x_test_tensor, y_train_tensor, y_test_tensor = train_test_split(
    x_tensor,
    y_tensor,
    test_size=.2,
    # random_state=4,
    shuffle=True,
);

In [265]:
%%time
logreg = LogisticRegression(
    n_jobs=-1,
    max_iter=500,
);
logreg.fit(x_train, y_train);
accuracy_score(y_test, logreg.predict(x_test))

CPU times: total: 15.6 ms
Wall time: 773 ms


0.8924298898642073

In [269]:
%%time
ranfor = RandomForestClassifier(
    n_estimators=230,
    n_jobs=-1,
    random_state=42021,
)
ranfor.fit(x_train, y_train)
accuracy_score(y_test, ranfor.predict(x_test))

CPU times: total: 7min 11s
Wall time: 1min 2s


0.9028070990616067

In [431]:
%%time
# selected
ranfortest = RandomForestClassifier(
    n_estimators=230,
    n_jobs=-1,
    random_state=42021,
)
ranfortest.fit(data_copy_scaled, data['loan_paid_back'])
# accuracy_score(y_test, ranfor.predict(x_test))

CPU times: total: 15min 13s
Wall time: 2min 18s


,n_estimators,230
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [423]:
# computational graph
in_features: int = 11
hidden_features: int = 256
out_features: int = 1

banker = nn.Sequential(
    nn.Linear(
        in_features=in_features,
        out_features=hidden_features,
    ),
    nn.ReLU(),
    nn.Linear(
        in_features=hidden_features,
        out_features=hidden_features,
    ),
    nn.ReLU(),
    nn.Linear(
        in_features=hidden_features,
        out_features=hidden_features,
    ),
    nn.ReLU(),
    nn.Linear(
        in_features=hidden_features,
        out_features=hidden_features,
    ),
    nn.ReLU(),
    nn.Linear(
        in_features=hidden_features,
        out_features=out_features,
    ),
);


In [432]:
def train_model(
    model: torch.nn,
    x_train: torch.tensor,
    y_train: torch.tensor,
    x_valid: torch.tensor,
    y_valid: torch.tensor,
    epochs: int,
    lossfunc: nn,
    optim: torch.optim,
) -> float: 
    for epoch in range(epochs):
        model.train()
        train_logits = model(x_train)
        train_loss = lossfunc(train_logits, y_train.unsqueeze(-1))
        optim.zero_grad()
        train_loss.backward()
        optim.step()
        train_accuracy = accuracy_score(y_train, torch.round(torch.sigmoid(train_logits)).detach().numpy())

        model.eval()
        with torch.inference_mode():
            valid_logits = model(x_valid)
            test_loss = lossfunc(valid_logits, y_valid.unsqueeze(-1))
            valid_accuracy = accuracy_score(y_valid, torch.round(torch.sigmoid(valid_logits)).detach().numpy())
        print('-', end=' ')
        if epoch % 10 == 0:
            print(epoch, " > ", train_accuracy, valid_accuracy , train_loss.detach(), test_loss)
    print("final - ",train_accuracy, valid_accuracy , train_loss.detach(), test_loss)          

In [428]:
loss_fn = nn.BCEWithLogitsLoss()
# optim = torch.optim.Adam(banker.parameters())
optim = torch.optim.SGD(banker.parameters(), lr=0.01)

In [429]:
train_model(
    model=banker,
    x_train=x_train_tensor,
    y_train=y_train_tensor,
    x_valid=x_test_tensor,
    y_valid=y_test_tensor,
    epochs=150,
    lossfunc=loss_fn,
    optim=optim
);

- 0  >  0.9022317154010459 0.9003106086751572 tensor(0.2567, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) tensor(0.2602)
- - - - - - - - - - 10  >  0.9022401329980324 0.9002180153031591 tensor(0.2566, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) tensor(0.2601)
- - - - - - - - - - 20  >  0.9022232978040594 0.9001338395104336 tensor(0.2566, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) tensor(0.2600)
- - - - - - - - - - 30  >  0.9022232978040594 0.9001590922482513 tensor(0.2566, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) tensor(0.2600)
- - - - - - - - - - 40  >  0.9022190890055661 0.9001759274067963 tensor(0.2566, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) tensor(0.2600)
- - - - - - - - - - 50  >  0.9022190890055661 0.9002264328824316 tensor(0.2566, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) tensor(0.2600)
- - - - - - - - - - 60  >  0.9022359241995391 0.9002180153031591 tensor(0.2566, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) tensor(0.2600)
- - - - - -

In [318]:
x_train_tensor.shape, y_train_tensor.size()

(torch.Size([296997, 11]), torch.Size([296997]))

# main test

In [27]:
test_path = 'playground-series-s5e11_unzipped/test.csv'

In [435]:
test_data = pd.read_csv(test_path)

In [437]:
data = test_data
data_copy = data.copy()
# data_copy.loc[data_copy['gender'] == 'Male']['gender'] = 1
# data_copy.loc[data_copy['gender'] == 'Female']['gender'] = 0
# data_copy['gender'][data_copy['gender'] == 'Male'] = np.float32(1) ;
# data_copy['gender'][data_copy['gender'] == 'Female'] = np.float32(0);

label_encoder_gen = LabelEncoder()
data_copy['gender'] = label_encoder_gen.fit_transform(data_copy['gender'])

label_encoder_marstatus = LabelEncoder()
data_copy['marital_status'] = label_encoder_marstatus.fit_transform(data_copy['marital_status'])

# data_copy['marital_status'][data_copy['marital_status'] == 'Single'] = np.float32(0) ;
# data_copy['marital_status'][data_copy['marital_status'] == 'Married'] = np.float32(1);
# data_copy['marital_status'][data_copy['marital_status'] == 'Divorced'] = np.float32(2);

lable_encoder_empstatus = LabelEncoder()
data_copy['employment_status'] = lable_encoder_empstatus.fit_transform(data_copy['employment_status'])
# data_copy['employment_status'][data_copy['employment_status'] == 'Self-employed'] = np.float32(0);
# data_copy['employment_status'][data_copy['employment_status'] == 'Employed'] = np.float32(1);
# data_copy['employment_status'][data_copy['employment_status'] == 'Unemployed'] = np.float32(2);

lable_encoder_loanpurp = LabelEncoder()
lable_encoder_loanpurp.fit(data_copy['loan_purpose'])
data_copy['loan_purpose'] = lable_encoder_loanpurp.transform(data_copy['loan_purpose'])
data_copy['loan_purpose'] = data_copy['loan_purpose'].apply(lambda x:np.float32(x))


ordinal_encoder_edu = OrdinalEncoder()
data_copy['education_level'] = ordinal_encoder_edu.fit_transform(pd.DataFrame(data_copy['education_level']))

ordinal_encoder_grade = OrdinalEncoder()
data_copy['grade_subgrade'] = ordinal_encoder_grade.fit_transform(pd.DataFrame(data_copy['grade_subgrade']))
test = True
data_copy.drop(columns=['id'], inplace=True);
# data_copy.drop(columns=['loan_paid_back'], inplace=True)
stan_scale = StandardScaler();
stan_scale.fit(data_copy)
data_copy_scaled = stan_scale.transform(data_copy)

In [438]:
test_data_copy_scaled = data_copy_scaled

In [439]:
preds = ranfortest.predict(test_data_copy_scaled)

In [ ]:
pred_df = pd.DataFrame(preds, columns=['loan_paid_back'])

In [446]:
final_df = pd.concat((test_data['id'], pred_df), axis=1)
final_df

,id,loan_paid_back
0,593994,1.0
1,593995,1.0
2,593996,0.0
3,593997,1.0
4,593998,1.0
...,...,...
254564,848558,1.0
254565,848559,1.0
254566,848560,1.0
254567,848561,1.0


In [450]:
final_df.to_csv('final_test_df.csv', index=False)

In [449]:
final_df

,id,loan_paid_back
0,593994,1.0
1,593995,1.0
2,593996,0.0
3,593997,1.0
4,593998,1.0
...,...,...
254564,848558,1.0
254565,848559,1.0
254566,848560,1.0
254567,848561,1.0


In [451]:
test_data_copy_scaled_tensor = torch.tensor(test_data_copy_scaled, dtype=torch.float32)

In [458]:
nn_logits = banker(test_data_copy_scaled_tensor)

In [461]:
nn_preds = torch.round(torch.sigmoid(nn_logits.detach())).numpy()

In [498]:
nn_pred_df = pd.concat((test_data['id'], pd.DataFrame(nn_preds, columns=['loan_paid_back'])), axis=1)

In [500]:
nn_pred_df.to_csv('final_df2.csv', index=False)